In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x2feffb0
Welcome to JupyROOT 6.28/10


In [2]:
%jsroot on

In [3]:
ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.ildStyle.SetOptStat(1)

In [5]:
%%cpp
using namespace ROOT::VecOps;

In [6]:
# df = ROOT.RDataFrame("events", "data/truejet/test/sw_sl/nano.root")
df = ROOT.RDataFrame("events", "data/truejet/test/ww_sl/nano.root")
df.Describe()

Dataframe from TChain events in file data/truejet/test/ww_sl/nano.root

Property                Value
--------                -----
Columns in total           36
Columns from defines        0
Event loops run             0
Processing slots            6

Column                          Type                                                            Origin
------                          ----                                                            ------
beamPol_e                       Int_t                                                           Dataset
beamPol_p                       Int_t                                                           Dataset
fCoordinates                    ROOT::Math::PxPyPzM4D<double>                                   Dataset
fCoordinates.fM                 Double_t                                                        Dataset
fCoordinates.fT                 Double_t                                                        Dataset
fCoordinates.fX      

In [7]:
# df = df.Range(100)

In [8]:

df = df.Define("l_MC_lvec_e", "l_MC_lvec.energy()")
df = df.Define("l_MC_lvec_theta", "l_MC_lvec.theta()")
df = df.Define("l_MC_lvec_cosTheta", "cos(l_MC_lvec_theta)")

df = df.Define("l_MC_lvec_pt", "l_MC_lvec.pt()")
df = df.Define("l_MC_lvec_pz", "l_MC_lvec.pz()")

df = df.Define("nu_MC_lvec_e", "nu_MC_lvec.energy()")

# pdg 11 is e-, -11 is e+
df = df.Define("l_MC_charge", "-1 * ROOT::Math::Sign(l_MC_pdg)")

### start of angle calculations
# the simplest is the production angle theta_W (where W stands not for Weinberg in this case but for W-boson)
# following M. Bilenky et al. in Nucl.Phys.B 409 (1993) 22-68 (doi: https://doi.org/10.1016/0550-3213(93)90445-U)
# this is the angle between the W^{-} and the beam e^{-} which in our case is just the Z-axis
# (TODO: check what happens when ddsim adds a crossing angle :/)

# check l charge to determine which ICN is W-
# TODO: normally the direction of the leptonically decaying W would need to be inferred from the hardronic one
df = df.Define("W_minus_lvec", "l_MC_charge == 1. ? qq_ICN_lvec : lnu_ICN_lvec")
df = df.Define("W_plus_lvec", "l_MC_charge == 1. ? lnu_ICN_lvec : qq_ICN_lvec")
df = df.Define("e_minus_vec", "ROOT::Math::XYZVector(0, 0, 1)")
df = df.Define("CosTheta_W_minus", "ROOT::Math::VectorUtil::CosTheta(W_minus_lvec, e_minus_vec)")

# build custom coordinate system t determine l angles:
# z1 points along the W- flight direction
df = df.Define("z1", "W_minus_lvec.Vect().Unit()")
# y1 is e_minus_vec x W dir/z1
df = df.Define("y1", "e_minus_vec.Cross(z1).Unit()")
# I *think* this is the right order..., no extra .Unit() should be necessary
df = df.Define("x1", "y1.Cross(z1)")
df = df.Define("rot1", "ROOT::Math::Rotation3D(x1, y1, z1)")
# the other coordinate system
df = df.Define("z2", "W_plus_lvec.Vect().Unit()")
# yes also e_minus here!
df = df.Define("y2", "e_minus_vec.Cross(z2).Unit()")
# I *think* this is the right order..., no extra .Unit() should be necessary
df = df.Define("x2", "y2.Cross(z2)")
df = df.Define("rot2", "ROOT::Math::Rotation3D(x2, y2, z2)")

# get boost
df = df.Define("W_minus_boost", "W_minus_lvec.BoostToCM()")
df = df.Define("W_plus_boost", "W_plus_lvec.BoostToCM()")

df = df.Define("l_boosted_lvec", "l_MC_charge == 1. ? ROOT::Math::VectorUtil::boost(l_MC_lvec, W_plus_boost) : ROOT::Math::VectorUtil::boost(l_MC_lvec, W_minus_boost)")
df = df.Define("l_transformed_lvec", "l_MC_charge == 1. ? rot2(l_boosted_lvec) : rot1(l_boosted_lvec)")

df = df.Define("l_theta_prime", "l_transformed_lvec.Theta()")
df = df.Define("l_phi_prime", "l_transformed_lvec.Phi()")
# df = df.Define("l_theta_prime", "auto theta = l_transformed_lvec.Theta(); return l_MC_charge == 1. ? ROOT::Math::Pi() - theta : theta;")
# df = df.Define("l_phi_prime", "auto phi = l_transformed_lvec.Phi(); return l_MC_charge == 1. ? phi - ROOT::Math::Pi() : phi;")

# OPAL shift W plus by 180 deg, but this gives it the wrong shape?!
# df = df.Define("l_phi_prime", "auto phi = l_transformed_lvec.Phi(); auto phi_s = l_MC_charge == 1. ? phi + ROOT::Math::Pi() : phi; return phi_s > ROOT::Math::Pi() ? phi_s - 2 * ROOT::Math::Pi() : phi_s;")

df = df.Define("l_cosTheta_prime", "cos(l_theta_prime)")

df = df.Define("lnu_ICN_m", "lnu_ICN_lvec.mass()")

In [9]:
# FIXME: cut on M_l,nu close to M_W for debugging
# df = df.Filter("abs(lnu_ICN_m - 80) < 5", "rough M_{l,nu} #approx M_{W}")
# FIXME: for debugging
# df = df.Redefine("weight", "1")
df = df.Filter("l_MC_lvec_pt > 2", "pt cut")
df = df.Filter("abs(l_MC_lvec_cosTheta) < 0.975", "cosTheta cut") # ~6.5 deg
# df = df.Filter("beamPol_e != beamPol_p")

In [10]:
h_l_e = df.Histo1D(("", ";E_{e} [GeV]", 300, 0., 150.), "l_MC_lvec_e", "weight")
h_nu_e = df.Histo1D(("", ";E_{#nu} [GeV]", 300, 0., 150.), "nu_MC_lvec_e", "weight")
h_2d_lnu_e = df.Histo2D(("", ";E_{e} [GeV];E_{#nu} [GeV]", 300, 0., 150., 300, 0., 150.), "l_MC_lvec_e", "nu_MC_lvec_e", "weight")
h_2d_lnu_pzpt = df.Histo2D(("", ";P_{z}(e) [GeV];P_{t}(e) [GeV]", 150, -150., 150., 150, 0., 150.), "l_MC_lvec_pz", "l_MC_lvec_pt", "weight")
h_2d_lnu_cosThetapt = df.Histo2D(("", ";cos #theta(e);P_{t}(e) [GeV]", 150, -1., 1., 150, 0., 150.), "l_MC_lvec_cosTheta", "l_MC_lvec_pt", "weight")

h_l_charge = df.Histo1D(("", ";q_{e}", 3, -1, 2), "l_MC_charge", "weight")
h_CosTheta_W_minus = df.Histo1D("CosTheta_W_minus", "weight")
# h_l_theta_prime = df.Histo1D("l_theta_prime", "weight")
h_l_cosTheta_prime = df.Histo1D("l_cosTheta_prime", "weight")
h_l_phi_prime = df.Histo1D("l_phi_prime", "weight")
# h_l_plus_theta_prime = df.Filter("l_MC_charge == 1.").Histo1D("l_theta_prime", "weight")
h_l_plus_cosTheta_prime = df.Filter("l_MC_charge == 1.").Histo1D("l_cosTheta_prime", "weight")
h_l_plus_phi_prime = df.Filter("l_MC_charge == 1.").Histo1D("l_phi_prime", "weight")
# h_l_minus_theta_prime = df.Filter("l_MC_charge == 1.").Histo1D("l_theta_prime", "weight")
h_l_minus_cosTheta_prime = df.Filter("l_MC_charge == -1.").Histo1D("l_cosTheta_prime", "weight")
h_l_minus_phi_prime = df.Filter("l_MC_charge == -1.").Histo1D(("", ";#phi*_{#ell}", 20, -ROOT.Math.Pi(), ROOT.Math.Pi()), "l_phi_prime", "weight")

# not what LT means in the paper
# h_CosTheta_W_minus_LT = df.Filter("beamPol_e == -1").Histo1D("CosTheta_W_minus", "weight")

In [11]:
c_l_e = ROOT.TCanvas()
h_l_e.Draw()
c_l_e.Draw()

c_nu_e = ROOT.TCanvas()
h_nu_e.Draw()
c_nu_e.Draw()

c_2d_lnu_e = ROOT.TCanvas()
h_2d_lnu_e.Draw("colz0")
c_2d_lnu_e.Draw()

c_2d_lnu_pzpt = ROOT.TCanvas()
h_2d_lnu_pzpt.Draw("colz0")
c_2d_lnu_pzpt.Draw()

c_2d_lnu_cosThetapt = ROOT.TCanvas()
h_2d_lnu_cosThetapt.Draw("colz0")
c_2d_lnu_cosThetapt.Draw()

c_l_charge = ROOT.TCanvas()
h_l_charge.Draw()
c_l_charge.Draw()

c_CosTheta_W_minus = ROOT.TCanvas()
h_CosTheta_W_minus.Draw()
c_CosTheta_W_minus.Draw()

# c_l_theta_prime = ROOT.TCanvas()
# h_l_theta_prime.Draw()
# c_l_theta_prime.Draw()

c_l_cosTheta_prime = ROOT.TCanvas()
h_l_cosTheta_prime.Draw()
c_l_cosTheta_prime.Draw()

c_l_phi_prime = ROOT.TCanvas()
h_l_phi_prime.Draw()
c_l_phi_prime.Draw()

# c_l_plus_theta_prime = ROOT.TCanvas()
# h_l_plus_theta_prime.Draw()
# c_l_plus_theta_prime.Draw()

c_l_plus_cosTheta_prime = ROOT.TCanvas()
h_l_plus_cosTheta_prime.Draw()
c_l_plus_cosTheta_prime.Draw()

c_l_plus_phi_prime = ROOT.TCanvas()
h_l_plus_phi_prime.Draw()
c_l_plus_phi_prime.Draw()

# c_l_minus_theta_prime = ROOT.TCanvas()
# h_l_minus_theta_prime.Draw()
# c_l_minus_theta_prime.Draw()

c_l_minus_cosTheta_prime = ROOT.TCanvas()
h_l_minus_cosTheta_prime.Draw()
c_l_minus_cosTheta_prime.Draw()

c_l_minus_phi_prime = ROOT.TCanvas()
h_l_minus_phi_prime.SetMinimum(0)
h_l_minus_phi_prime.Draw()
c_l_minus_phi_prime.Draw()

# c_CosTheta_W_minus_LT = ROOT.TCanvas()
# h_CosTheta_W_minus_LT.Draw()
# c_CosTheta_W_minus_LT.Draw()